In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from datetime import datetime, time
import glob
from autogluon.tabular import TabularDataset, TabularPredictor
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from tqdm import tqdm

In [2]:
def display_errors(y_true, y_pred, return_vals=False):
    mse = mean_squared_error(y_true, y_pred)
    print(f"MSE: {round(mse, 3)}")
    rmse = mse ** 0.5
    print(f"RMSE: {round(rmse, 3)}")
    mae = mean_absolute_error(y_true, y_pred)
    print(f"MAE: {round(mae, 3)}")
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f"MAPE: {round(mape, 3)}")
    r2 = r2_score(y_true, y_pred)
    print(f"R2: {round(r2, 3)}")

    if return_vals:
        return [mse, rmse, mae, mape, r2]

def plot_errors(y_true, y_pred, name='N/A'):
    plt.scatter(y_true, y_pred, color='blue', marker='o', label=f'error adjusted predictions ({name}) vs actual degree days')
    plt.xlabel('actual degree days')
    plt.ylabel('predicted degree days')
    plt.title(f'Prediction vs Actual ({name})')

    max_value = max(max(y_true[-225:]), max(y_true[-225:]))
    min_value = min(min(y_true[-225:]), min(y_true[-225:]))
    plt.plot([min_value, max_value], [min_value, max_value], color='red', linestyle='--', label='Line of Equality')

    plt.legend()

In [3]:
class LinReg:
    def __init__(self, train_data, test_data, target_col):
        self.X_train = train_data.drop(columns=[target_col])
        self.y_train = train_data[target_col]
        self.X_test = test_data.drop(columns=[target_col])
        self.y_test = test_data[target_col]
        self.target_col = target_col
        self.model = LinearRegression().fit(self.X_train, self.y_train)
        self.y_pred = self.model.predict(self.X_test)
        self.evaluate()

    def evaluate(self):
        display_errors(self.y_test, self.y_pred)


In [4]:
def extract_date_time(filename):
    """
    extract the date and time from the filename
    :param filename:
    :return:
    """
    parts = filename.split('.')
    extracted_date = parts[1]
    extracted_time = parts[2]
    return extracted_date, extracted_time

def get_date(file):
        """get the date from the dataframe and the time from the filename and combine them into a datetime object
        :param file: filename containing the time
        :return: datetime object
        """

        date_str = str(file.split('.')[1])
        time_str = str(file.split('.')[2])
        date_value = datetime.strptime(date_str, '%Y%m%d')
        time_value = time(int(time_str), 0)
        combined_datetime = datetime.combine(date_value.date(), time_value)
        return combined_datetime
#give type hint on start_date that it should be a string
def seasonal_train_test(data, start_date: str, end_date: str):
    """
    split the data into train and test sets based on dates
    :param data: dataframe containing the data
    :param start_date: start date of the test set
    :param end_date: end date of the test set
    :return: train and test sets
    """

    test_start = start_date
    test_end = end_date
    test_data = data.loc[test_start: test_end]

    #remove X_test from dataframe to get X_train
    mask = (data.index >= test_start) & (data.index <= test_end)
    train_data = data.loc[~mask]

    return train_data, test_data

In [5]:
class RawDataProcess:
    def __init__(self, degree_days='gw_hdd', path='RawData', time=None):
        self.degree_days = degree_days
        self.path = path
        self.sort_files()
        self.get_master_model()
        if time is not None:
            self.filter_time(time)


    def filter_time(self, time):
        data = self.master_data
        data['hour'] = data.index.hour
        data = data[data['hour'] == time]
        data.drop('hour', axis=1, inplace=True)
        self.master_data = data

    def sort_files(self):
        """
        sort the files in the directory by date and time
        :return:
        """
        degree_days = self.degree_days
        path = self.path

        degree_days = degree_days
        ecmwf_files = glob.glob(path + f'/ecmwf.*.[01][02].{degree_days}.csv')
        ecmwf_sorted_files = sorted(ecmwf_files, key=lambda x: (x.split('.')[1], x.split('.')[2]))[3:]

        ecmwf_ens_files = glob.glob(path + f'/ecmwf-eps.*.[01][02].{degree_days}.csv')
        ecmwf_ens_sorted_files = sorted(ecmwf_ens_files, key=lambda x: (x.split('.')[1], x.split('.')[2]))[2:]

        gfs_ens_bc_files = glob.glob(path + f'/gfs-ens-bc.*.[01][02].{degree_days}.csv')
        gfs_ens_bc_sorted_files = sorted(gfs_ens_bc_files, key=lambda x: (x.split('.')[1], x.split('.')[2]))[2:]

        cmc_ens_files = glob.glob(path + f'/cmc-ens.*.[01][02].{degree_days}.csv')
        cmc_ens_sorted_files = sorted(cmc_ens_files, key=lambda x: (x.split('.')[1], x.split('.')[2]))[2:]
        for _ in range(2):
            set1 = set((extract_date_time(filename) for filename in ecmwf_sorted_files))
            set2 = set((extract_date_time(filename) for filename in ecmwf_ens_sorted_files))

            ecmwf_sorted_files = [filename for filename in ecmwf_sorted_files if extract_date_time(filename) in set2]
            ecmwf_ens_sorted_files = [filename for filename in ecmwf_ens_sorted_files if
                                      extract_date_time(filename) in set1]
            cmc_ens_sorted_files = [filename for filename in cmc_ens_sorted_files if extract_date_time(filename) in set1]

            master_set = set((extract_date_time(filename) for filename in cmc_ens_sorted_files))
            gfs_ens_bc_sorted_files = [filename for filename in gfs_ens_bc_sorted_files if
                                       extract_date_time(filename) in master_set]

            master_set = set((extract_date_time(filename) for filename in gfs_ens_bc_sorted_files))

            ecmwf_sorted_files = [filename for filename in ecmwf_sorted_files if extract_date_time(filename) in master_set]
            ecmwf_ens_sorted_files = [filename for filename in ecmwf_ens_sorted_files if
                                      extract_date_time(filename) in master_set]
            gfs_ens_bc_sorted_files = [filename for filename in gfs_ens_bc_sorted_files if
                                       extract_date_time(filename) in master_set]
            cmc_ens_sorted_files = [filename for filename in cmc_ens_sorted_files if
                                    extract_date_time(filename) in master_set]

        self.ecmwf_sorted_files = ecmwf_sorted_files
        self.ecmwf_ens_sorted_files = ecmwf_ens_sorted_files
        self.gfs_ens_bc_sorted_files = gfs_ens_bc_sorted_files
        self.cmc_ens_sorted_files = cmc_ens_sorted_files

    def y_value(self, start=8, end=14):
        ecmwf_ens_9_14 = pd.DataFrame(columns=[f'ens({start+1},{end})'])

        ecmwf_ens_sorted_files = self.ecmwf_ens_sorted_files

        for i in range(1, len(ecmwf_ens_sorted_files)):
            ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i])
            ecmwf_ens_df = ecmwf_ens_df[ecmwf_ens_df[ecmwf_ens_df.columns[2]] >= 1]
            prev_ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i - 1])
            prev_ecmwf_ens_df = prev_ecmwf_ens_df[prev_ecmwf_ens_df[prev_ecmwf_ens_df.columns[2]] >= 1]

            date = get_date(ecmwf_ens_sorted_files[i])
            prev_date = get_date(ecmwf_ens_sorted_files[i - 1])
            d2 = str(date)[:10]
            d1 = str(prev_date)[:10]

            if d2 == d1:
                offset = 1
            else:
                offset = 0

            cur = ecmwf_ens_df['Value'].iloc[start:end].sum()
            prev = prev_ecmwf_ens_df['Value'].iloc[(start+offset):(end+offset)].sum()
            change = cur - prev

            new_row = pd.DataFrame(change, columns=ecmwf_ens_9_14.columns, index=[date])
            ecmwf_ens_9_14 = pd.concat([ecmwf_ens_9_14, new_row])

        self.y_values = ecmwf_ens_9_14

    def ecmwf_ens(self, start=7, end=8):
        ecmwf_ens_8 = pd.DataFrame(columns=[f'ens({end})'])
        ecmwf_ens_sorted_files = self.ecmwf_ens_sorted_files

        for i in range(1, len(ecmwf_ens_sorted_files)):
            ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i])
            ecmwf_ens_df = ecmwf_ens_df[ecmwf_ens_df[ecmwf_ens_df.columns[2]] >= 1]
            prev_ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i - 1])
            prev_ecmwf_ens_df = prev_ecmwf_ens_df[prev_ecmwf_ens_df[prev_ecmwf_ens_df.columns[2]] >= 1]

            date = get_date(ecmwf_ens_sorted_files[i])
            prev_date = get_date(ecmwf_ens_sorted_files[i - 1])
            d2 = str(date)[:10]
            d1 = str(prev_date)[:10]

            if d2 == d1:
                offset = 1
            else:
                offset = 0

            cur = ecmwf_ens_df['Value'].iloc[start:end].sum() #7-8 benchmark, 7-8 best results
            prev = prev_ecmwf_ens_df['Value'].iloc[(start+offset):(end+offset)].sum() #7-8 benchmark, 7-8 best results
            change = cur - prev

            new_row = pd.DataFrame(change, columns=ecmwf_ens_8.columns, index=[date])
            ecmwf_ens_8 = pd.concat([ecmwf_ens_8, new_row])

        self.ecmwf_ens_data = ecmwf_ens_8

    def ecmwf(self, start=8, end=9):

        ecmwf_9_10 = pd.DataFrame(columns=[f'ecmwf({end})'])

        ecmwf_sorted_files = self.ecmwf_sorted_files
        ecmwf_ens_sorted_files = self.ecmwf_ens_sorted_files


        for i in range(1, len(ecmwf_sorted_files)):
            ecmwf_df = pd.read_csv(ecmwf_sorted_files[i])
            ecmwf_df = ecmwf_df[ecmwf_df[ecmwf_df.columns[2]] >= 1]
            prev_ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i-1])
            prev_ecmwf_ens_df = prev_ecmwf_ens_df[prev_ecmwf_ens_df[prev_ecmwf_ens_df.columns[2]] >= 1]

            date = get_date(ecmwf_sorted_files[i])
            prev_date = get_date(ecmwf_sorted_files[i - 1])
            d2 = str(date)[:10]
            d1 = str(prev_date)[:10]

            if d2 == d1:
                offset = 1
            else:
                offset = 0

            cur = ecmwf_df['Value'].iloc[start:end].sum() #8-9 benchmark, 4-9 best results
            prev = prev_ecmwf_ens_df['Value'].iloc[(start+offset):(end+offset)].sum() #8-9 benchmark, 4-9 best results

            change = cur - prev

            new_row = pd.DataFrame(change, columns=ecmwf_9_10.columns, index=[date])
            ecmwf_9_10 = pd.concat([ecmwf_9_10, new_row])

        self.ecmwf_data = ecmwf_9_10

    def gfs(self, start=9, end=14):
        gfs_11_14 = pd.DataFrame(columns=[f'gfs({start+1},{end})'])

        ecmwf_ens_sorted_files = self.ecmwf_ens_sorted_files
        gfs_ens_bc_sorted_files = self.gfs_ens_bc_sorted_files

        for i in range(1, len(gfs_ens_bc_sorted_files)):
            gfs_df = pd.read_csv(gfs_ens_bc_sorted_files[i])
            gfs_df = gfs_df[gfs_df[gfs_df.columns[2]] >= 1]
            prev_ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i-1])
            prev_ecmwf_ens_df = prev_ecmwf_ens_df[prev_ecmwf_ens_df[prev_ecmwf_ens_df.columns[2]] >= 1]

            date = get_date(gfs_ens_bc_sorted_files[i])
            prev_date = get_date(ecmwf_ens_sorted_files[i - 1])
            d2 = str(date)[:10]
            d1 = str(prev_date)[:10]

            if d2 == d1:
                offset = 1
            else:
                offset = 0

            cur = gfs_df['Value'].iloc[start:end].sum() # 9-14 benchmark, 9-16 best results
            prev = prev_ecmwf_ens_df['Value'].iloc[(start+offset):(end+offset)].sum() # 9-14 benchmark, 9-16 best results

            change = cur - prev

            new_row = pd.DataFrame(change, columns=gfs_11_14.columns, index=[date])
            gfs_11_14 = pd.concat([gfs_11_14, new_row])

        self.gfs_data = gfs_11_14

    def cmc(self, start=8, end=14):
        cmc_9_14 = pd.DataFrame(columns=[f'cmc({start+1},{end})'])

        cmc_ens_sorted_files = self.cmc_ens_sorted_files
        gfs_ens_bc_sorted_files = self.gfs_ens_bc_sorted_files

        for i in range(1, len(cmc_ens_sorted_files)):
            cmc_df = pd.read_csv(cmc_ens_sorted_files[i])
            cmc_df = cmc_df[cmc_df[cmc_df.columns[2]] >= 1]
            gfs_df = pd.read_csv(gfs_ens_bc_sorted_files[i])
            gfs_df = gfs_df[gfs_df[gfs_df.columns[2]] >= 1]

            date = get_date(cmc_ens_sorted_files[i])

            cmc = cmc_df['Value'].iloc[start:end].sum() #8-14 benchmark, 8-14 best results
            gfs = gfs_df['Value'].iloc[start:end].sum() #8-14 benchmark, 8-14 best results
            change = cmc - gfs

            new_row = pd.DataFrame(change, columns=cmc_9_14.columns, index=[date])
            cmc_9_14 = pd.concat([cmc_9_14, new_row])

        self.cmc_data = cmc_9_14

    def norm(self):
        norms = pd.DataFrame(columns=['Date', 'Value'])
        ecmwf_ens_sorted_files = self.ecmwf_ens_sorted_files

        for i in range(1, len(ecmwf_ens_sorted_files), 2):
            ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i])
            v1 = ecmwf_ens_df[ecmwf_ens_df[ecmwf_ens_df.columns[2]] == 2].iloc[:, :2]
            norms = pd.concat([norms, v1]).drop_duplicates('Date')

        norms.reset_index(inplace=True)
        norms.drop(columns=['index'], inplace=True)
        norms['Date'] = pd.to_datetime(norms['Date']).dt.strftime('%Y-%m-%d 12:00:00')
        norms.set_index('Date', inplace=True)
        norms.rename_axis('', inplace=True)
        norms.rename(columns={'Value': 'norm'}, inplace=True)

        self.norms_data = norms

    def run_all_models(self):
        self.y_value()
        self.ecmwf_ens()
        self.ecmwf()
        self.gfs()
        self.cmc()
        self.norm()

    def get_master_model(self):
        self.run_all_models()
        master_data = pd.concat([self.ecmwf_ens_data, self.ecmwf_data,
                                 self.gfs_data, self.cmc_data, self.y_values], axis=1)

        self.master_data = master_data

In [6]:
data = RawDataProcess(degree_days='gw_hdd', path='RawData', time=12)

/var/folders/3y/4qthl0m139n7983g0dhsq1n00000gn/T/ipykernel_21275/2723095553.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop('hour', axis=1, inplace=True)


In [28]:
master_data = data.master_data

In [34]:
train_data, test_data = seasonal_train_test(master_data, '2022-10-04 12:00:00', '2023-5-16 12:00:00')

In [35]:
train_data = TabularDataset(train_data)
test_data = TabularDataset(test_data)

In [36]:
label = 'ens(9,14)'

In [37]:
y_test = test_data[label]
test_data_nolabel = test_data.drop(columns=[label])

In [38]:
predictor = TabularPredictor(label=label).fit(train_data) # presets='best_quality'

No path specified. Models will be saved in: "AutogluonModels/ag-20230711_211037/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230711_211037/"
AutoGluon Version:  0.7.0
Python Version:     3.10.9
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 22.5.0: Thu Jun  8 22:22:19 PDT 2023; root:xnu-8796.121.3~7/RELEASE_ARM64_T8103
Train Data Rows:    1524
Train Data Columns: 1334
Label Column: ens(9,14)
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (18.34899999999999, -21.283999999999992, 0.37736, 3.97246)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...


In [39]:
fi = predictor.feature_importance(test_data, silent=True)

These features in provided data are not utilized by the predictor and will be ignored: ['tavg_72280', 'tmin_72280', 'tmax_72280', 'prcp_72280', 'pres_72280']


In [40]:
display(fi.head(10))

,importance,stddev,p_value,n,p99_high,p99_low
ecmwf(9),0.245170,0.057802,0.000345,5,0.364185,0.126155
"gfs(10,14)",0.220840,0.044485,0.000187,5,0.312435,0.129245
ens(8),0.129705,0.034565,0.000552,5,0.200875,0.058535
tavg_OZNF7,0.010330,0.002391,0.000321,5,0.015252,0.005407
pres_LQO8Z,0.007343,0.005331,0.018470,5,0.018320,-0.003634
tmin_72266,0.006481,0.002440,0.002015,5,0.011504,0.001457
tavg_KPPF0,0.005884,0.003091,0.006546,5,0.012249,-0.000481
pres_72775,0.005812,0.003118,0.007026,5,0.012232,-0.000608
prcp_KRNM0,0.005292,0.001866,0.001584,5,0.009135,0.001450
tmin_72775,0.005169,0.002762,0.006932,5,0.010857,-0.000518


In [41]:
y_pred = predictor.predict(test_data_nolabel)
display_errors(y_test, y_pred)

MSE: 18.139
RMSE: 4.259
MAE: 3.222
MAPE: 3.359
R2: 0.143


In [26]:
linreg = LinReg(train_data, test_data, label)

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

# weather data

In [8]:
from meteostat import Stations, Daily

In [9]:
stations = pd.read_csv('StormVistaData/station_df.csv')

In [10]:
stations.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

In [11]:
station_ids = stations['id'].tolist()

In [12]:
start = datetime(2018, 7, 11)
end = datetime(2023, 6, 16)

In [27]:
# Get daily data
weather_df = pd.DataFrame()

for i in tqdm(station_ids[:266]):
    weather_data = Daily(i, start, end)
    weather_data = weather_data.fetch()
    weather_data = weather_data[['tavg', 'tmin', 'tmax', 'prcp', 'pres']]
    weather_data = weather_data.diff(-15)
    weather_data = weather_data.add_suffix(f'_{i}')
    weather_df = pd.concat([weather_df, weather_data], axis=1)

100%|██████████| 266/266 [00:02<00:00, 107.63it/s]


In [14]:
display(weather_df)

,tavg_72529,tmin_72529,tmax_72529,prcp_72529,pres_72529,tsun_72529,tavg_TMT7J,tmin_TMT7J,tmax_TMT7J,prcp_TMT7J,...,tmax_72405,prcp_72405,pres_72405,tsun_72405,tavg_72514,tmin_72514,tmax_72514,prcp_72514,pres_72514,tsun_72514
time,,,,,,,,,,,,,,,,,,,,,
2018-07-11,-0.8,-2.7,-4.4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2.7,0.0,1.3,NaN,-0.8,-3.9,-1.2,0.0,4.6,NaN
2018-07-12,-2.0,-5.0,2.8,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.5,0.0,5.4,NaN,-1.5,-7.2,3.3,-0.5,7.6,NaN
2018-07-13,3.9,1.1,6.1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.1,0.0,6.6,NaN,-1.0,-2.8,1.7,0.0,6.8,NaN
2018-07-14,6.2,7.3,6.7,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2.2,0.0,1.3,NaN,3.4,3.9,4.4,0.0,-0.8,NaN
2018-07-15,4.8,4.4,6.1,-0.3,NaN,NaN,NaN,NaN,NaN,NaN,...,5.5,-13.0,-2.4,NaN,6.1,6.1,7.3,6.1,-4.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
weather_df.reset_index(inplace=True)
weather_df['Date'] = pd.to_datetime(weather_df['time']).dt.strftime('%Y-%m-%d 12:00:00')
weather_df.drop(columns=['time'], inplace=True)
weather_df.set_index('Date', inplace=True)
weather_df.rename_axis('', inplace=True)
weather_df.index = pd.to_datetime(weather_df.index)

In [30]:
common_dates = master_data.index.intersection(weather_df.index)
master_data = pd.concat([master_data.loc[common_dates], weather_df.loc[common_dates]], axis=1)

# ------------------

In [31]:
display(master_data)

,ens(8),ecmwf(9),"gfs(10,14)","cmc(9,14)","ens(9,14)",tavg_72529,tmin_72529,tmax_72529,prcp_72529,pres_72529,...,tavg_72405,tmin_72405,tmax_72405,prcp_72405,pres_72405,tavg_72514,tmin_72514,tmax_72514,prcp_72514,pres_72514
2018-07-11 12:00:00,0.003,0.024,0.039,-0.040,0.004,-0.8,-2.7,-4.4,0.0,NaN,...,4.2,2.8,2.7,0.0,1.3,-0.8,-3.9,-1.2,0.0,4.6
2018-07-12 12:00:00,-0.002,0.005,0.046,-0.053,0.001,-2.0,-5.0,2.8,0.0,NaN,...,0.9,-1.1,0.5,0.0,5.4,-1.5,-7.2,3.3,-0.5,7.6
2018-07-13 12:00:00,0.000,0.003,0.037,-0.048,0.001,3.9,1.1,6.1,0.0,NaN,...,0.0,-0.5,1.1,0.0,6.6,-1.0,-2.8,1.7,0.0,6.8
2018-07-14 12:00:00,-0.001,-0.001,0.046,-0.053,0.000,6.2,7.3,6.7,0.0,NaN,...,1.0,0.0,2.2,0.0,1.3,3.4,3.9,4.4,0.0,-0.8
2018-07-15 12:00:00,0.000,0.006,0.034,-0.039,0.012,4.8,4.4,6.1,-0.3,NaN,...,2.7,1.1,5.5,-13.0,-2.4,6.1,6.1,7.3,6.1,-4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-12 12:00:00,-1.098,-1.852,0.549,-2.337,0.378,4.7,5.5,3.9,0.0,-8.7,...,2.2,2.8,3.8,0.0,-6.5,3.1,3.8,3.9,0.0,-7.3
2023-05-13 12:00:00,-0.563,-0.920,1.340,0.595,-0.801,1.5,2.8,-3.9,0.0,0.2,...,2.1,2.2,0.5,3.0,-1.6,2.1,3.4,-1.6,0.0,-1.3
2023-05-14 12:00:00,0.210,-0.419,1.168,-2.417,0.524,-5.2,-2.7,-10.5,0.0,10.5,...,-0.3,-2.2,1.7,-21.8,6.8,-5.4,-3.9,-10.0,0.0,8.7
2023-05-15 12:00:00,-0.080,1.089,1.291,-2.618,1.045,-7.9,-7.8,-7.2,0.0,3.6,...,-0.1,-1.7,-0.5,-1.0,3.4,-6.8,-8.4,-3.4,0.0,2.4
